# Gradients: The parameter-shift rule
#### Challenge (3/15)
#### 100 points

There are many ways of differentiating quantum circuits. One such way that is also hardware-compatible (i.e., it can be computed using a real quantum computer) is the parameter-shift rule. Here, we'll explore using the parameter-shift rule in the simplest case: for differentiating the Pauli rotation gates.

Consider a circuit given by a unitary $U(\vec{\theta})$, where $\vec{\theta}=(\theta_1, ..., \theta_n)$ are real differentiable parameters, whose output is an expectation value with respect to an observable $A$:

$$f(\vec{\theta}) = \left< 0 \right| U^\dagger(\vec{\theta})AU(\vec{\theta}) \left| 0 \right>$$

If the parameterized gates in $U(\vec{\theta})$ were only composed of Pauli rotation gates ($RX$, $RY$, and $RZ$) then differentiating $f(\vec{\theta})$ with respect to one of the parameters can be shown to simplify to

where $s$ is a finite-valued _shift_. Although this looks eerily similar to differentiation via finite-differences, $s$ need not be infinitesimally small! In this problem, you will implement the parameter-shift rule for the Pauli rotation gates yourself.

### Challenge code

In the provided code, you are given a couple of functions:
* `circuit`: This describes the function $f(\vec{\theta})$.
* `my_parameter_shift_grad`: This is where you will implement calculating $\frac{\partial}{\partial \theta_i}f(\vec{\theta}$ for every $\theta_i \in \vec{\theta}$.
It will return the gradient, which is a vector containing every partial derivative. **You must complete this function**.

### Input

As input to this problem, you are given:

* `params` (`list(float)`): a list corresponding to $\vec{\theta}$
* `shift` (`float`): the shift amount, corresponding to $s$.

### Output

This code will output the `gradient` (`list(float)`) of the `circuit`.

If your solution matches the correct one within the given tolerance specified in `check` (in this case it's a `1e-4` relative error tolerance), the output will be `"Correct!"`. Otherwise, you will receive a `"Wrong answer"` prompt.

Good luck!

### Code

In [1]:
import json

import pennylane as qml
import pennylane.numpy as np

In [2]:
dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev)
def circuit(params):
    """The quantum circuit that you will differentiate!

    Args:
        params (list(float)): The parameters for gates in the circuit

    Returns:
        (numpy.array): An expectation value.
    """
    qml.RY(params[0], 0)
    qml.RX(params[1], 1)
    return qml.expval(qml.PauliZ(0) + qml.PauliZ(1))

In [3]:
def my_parameter_shift_grad(params, shift):
    """Your homemade parameter-shift rule function.
    
    NOTE: you cannot use qml.grad within this function

    Args:
        params (list(float)): The parameters for gates in the circuit

    Returns:
        gradient (numpy.array): The gradient of the circuit with respect to the given parameters.
    """

    def shift_vector(vector, pos, shift):
        """This function creates the "shifted parameters" to feed in the circuit

        Args:
            vector (list(float)): The parameter vector to shift
            pos (int): The component to shift
            shift (float) : Value of the shift

        Return:
            shifted_vectors : (Tuple) Vectors of with opposite shifts
        """
        vector_plus  = np.copy(vector)
        vector_minus = np.copy(vector)

        vector_plus[pos]  += shift 
        vector_minus[pos] -= shift

        return (vector_plus, vector_minus)

    gradient = np.zeros_like(params)

    for i in range(len(params)):
        shift_plus, shift_minus = shift_vector(params, i, shift)
        gradient[i] = (circuit(shift_plus) - circuit(shift_minus) )

    gradient = gradient / (2*np.sin(shift))
    print('GRAD', gradient)
    return np.round_(gradient, decimals=5).tolist()
    

In [4]:
def run(test_case_input: str) -> str:
    params, shift = json.loads(test_case_input)
    gradient = my_parameter_shift_grad(params, shift)
    return str(gradient)

def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(
        solution_output, expected_output, rtol=1e-4
    ), "Your gradient isn't quite right!"


test_cases = [['[[0.75, 1.0], 1.23]', '[-0.68164, -0.84147]']]

for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")


Running test case 0 with input '[[0.75, 1.0], 1.23]'...
GRAD [-0.68163876 -0.84147098]
Correct!
